# Transformers Text Classification

In [61]:
import transformers
import pandas as pd
import torch
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [2]:
dir_path = "/mnt/c/Users/FEDERICO/Desktop/Federico/Formazione/MNLP/HW1/"
train_path = "train.csv"
dev_path = "valid.csv"

train_df = pd.read_csv(train_path, encoding='utf-8')
dev_df = pd.read_csv(dev_path, encoding='utf-8')

In [3]:
def save_txt(filename, path, txt):
    with open(path + filename, 'w', encoding='utf-8') as output:
        json.dump(txt, output, ensure_ascii=False, indent=2)

def load_txt(filename, path):
    with open(path + filename, 'r', encoding='utf-8') as input_file:
        return json.load(input_file)

train_txt = load_txt(filename="train_paragraphs.txt", path="./")
valid_txt = load_txt(filename="dev_paragraphs.txt", path="./")

train_df['paragraph'] = train_txt
dev_df['paragraph'] = valid_txt

In [76]:
model_name = "albert-large-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_df['paragraph'].to_list(), truncation=True, padding=True, max_length=1000)
val_encodings = tokenizer(dev_df['paragraph'].to_list(), truncation=True, padding=True, max_length=1000)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

In [77]:
mapper = {
    'cultural agnostic': 2,
    'cultural representative': 1,
    'cultural exclusive': 0
}

class PLMDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = [mapper[label] for label in labels]

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]).to("cuda") for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx]).to("cuda")
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PLMDataset(train_encodings, train_df['label'])
val_dataset = PLMDataset(val_encodings, dev_df['label'])

In [78]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to("cuda")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/71.5M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-large-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=1024, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_features=1024, out_features=1024, bias=True)
                (value): Linear(in_features=1024, out_fe

In [91]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from transformers import EarlyStoppingCallback

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="none",
    eval_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()